# Project Motivation
Metadistributions as discussed in [Keelin (2016)](http://www.metalogdistributions.com/images/The_Metalog_Distributions_-_Keelin_2016.pdf) and earlier papers of his essentially result from expanding the parameters of a distribution as a power series. By choosing carefully which distributions to use it is possible create distributions with high shape flexibility. Keelin initially chooses to use the logistic distribution because its quantile function
$$Q(y;\mu,s) = \mu + s ln\left( \frac{y}{1-y} \right), 0<y<1$$

is linear in its parameters, where $\mu$ is the mean, median, and mode, and $s$ is proportional to the standard deviation $\sigma = \frac{s \pi}{\sqrt{3}}$. Following expansion of $\mu$ and $s$ in cumulative probability with real parameters $a_i$

$$\mu = a_1 + a_4 (y - 0.5) + a_5 (y - 0.5)^2 + a_7 (y - 0.5)^3 + a_9 (y - 0.5)^4 + ...,$$
$$s = a_2 + a_3 (y - 0.5) + a_6 (y - 0.5)^2 + a_8 (y - 0.5)^3 + a_{10}(y - 0.5)^4 + ...,$$

we may use linear least squares to fit the $a_i$'s to data. As additional terms are added to the expansions, the shape flexibility of the metalogistic distribution quickly approaches the upper limit for all distributions.

The shape flexibility given by this methodology is potentially useful within mathematical finance for two main reasons. 

Attention has been increasingly paid to the fact that many quantities of interest which describe the behaviour of financial markets over the long term have fat tailed distributions. Given that probabilistic models which utilize moment based descriptions - such as the standard deviation for risk measurment - are highly sensitive to the distributional tails, having a precisely tuned tail to the distribution within these models can make a large difference to both their utility and their viability.

The parameter based shape flexibility can be also used to create a family of information manifolds. A practical application of this is the usage of recurrent neural networks to continuously vary the parameters of the distribution across a period of time to allow the generation of stochastic processes with wildly different characteristics and arbitrary dependence structures provided by the maintainence of long term dependence within recurrent neural networks such as long short-term memory networks.

It is from these and other motivations I have pursued in some time outside my studies to the construction of multivariate metadistributions, one of which is shown below.


# Current Progress

The single variable Gumbel distribution has CDF

$$F(x,\mu,\beta) = \exp(-\exp(-(x - \mu)/\beta))$$

which one of the Gumbel distributions multivariate generalisations may have a CDF taking the form

$$F(x,\mu,\beta) = \exp \left( \sum_{i} -\exp \left( -(x_{i} - \mu_{i})/\beta_{i} \right) \right)$$

So far my investigations have indicated that the luxury of using linear least squares to fit the coefficients of the parameter expansions does not generalise to the mulitvariate setting as can be seen in the formulae for the CDFs of most multivarite distributions, including the above multivariate Gumbel CDF. I continue by instead using nonlinear optimisation techniques, of which I have found success with particle swarm optimisation.

This rather simple multivariate Gumbel has difficulties when fitting its expanded moments around the origin as the polynomials are double exponentiated. I am currently in the process of exploring other distributions which may be more suitable as a base for an unbounded multivariate metadistribution. As a result I have tested it primarily on semi-bounded distributions, including these three tests on jointly independent semibounded tailed distributions.

### The fitting process is observed as additional draws from the distributions are made.

![image](e_metalog_scatter.gif)
![image](m_metalog_scatter.gif)
![image](p_metalog_scatter.gif)

Of initial note is how quickly the error of fit in the tails of the double $Exponential(\lambda = 2)$ vanishes. The error surface displayed here begins at $(20,20)$ which is at the $~99.75$ percentile for this distribution. In the region $ 20 < X < 40$ , $20 < Y < 40$ the error after $n = 60$ is close to $10^{-4}$ 

Despite the inherently more difficult to fit Pareto distribution we can see in its near tail a relatively close convergence to the true distributional values.

These beginning results are being used to direct my further investigation into methods for allowing smoother manipulation with faster optimisation and closer fits.

In [ ]:
using Distributions, Plots, Flux, TensorOperations, LinearAlgebra, MultinomialSeries, Optim

In [ ]:
#
# Functions for calculation of tensor eigenvalues gathered from the library TensorFactorizations.jl
#

In [ ]:
function tensoreig(A, a, b; chis=nothing, eps=0, return_error=false,
                   print_error=false, break_degenerate=false,
                   degeneracy_eps=1e-6, norm_type=:frobenius,
                   hermitian=false)
    # Create the matrix and decompose it.
    A, shp_a, shp_b = to_matrix(A, a, b; return_tensor_shape=true)
    if hermitian
        A = Hermitian((A+A')/2)
    end
    fact = eigen(A)
    E, U = fact.values, fact.vectors
    # Sort the by largest magnitude eigenvalue first.
    perm = sortperm(abs.(E), rev=true)
    if perm != collect(1:length(E))
        E = E[perm]
        U = U[:,perm]
    end

    # Find the dimensions to truncate to and the error caused in doing so.
    chi, error = find_trunc_dim(E, chis, eps, break_degenerate, degeneracy_eps,
                                norm_type)

    # Truncate
    E = E[1:chi]
    U = U[:,1:chi]

    if print_error
        println("Relative truncation error ($norm_type norm) in eig: $error")
    end

    # Reshape U to a tensor with a shape matching the shape of A and
    # return.
    dim = size(E)[1]
    U_tens = reshape(U, shp_a..., dim)
    retval = (E, U_tens)
    if return_error
        retval = (retval..., error)
    end
    return retval
end


function to_matrix(A, a, b; return_tensor_shape=false)
    # Make sure a and b are Arrays.
    if isa(a, Number)
        a = [a]
    elseif !isa(a, Array)
        a = collect(a)
    end
    if isa(b, Number)
        b = [b]
    elseif !isa(b, Array)
        b = collect(b)
    end

    # Permute the indices of A to the right order
    perm = vcat(a, b)
    A = tensorcopy(A, collect(1:ndims(A)), perm)
    # The lists shp_a and shp_b list the dimensions of the bonds in a and b
    shp = size(A)
    shp_a = shp[1:length(a)]
    shp_b = shp[end+1-length(b):end]

    # Compute the dimensions of the the matrix that will be formed when
    # indices of a and b are joined together.
    dim_a = prod(shp_a)
    dim_b = prod(shp_b)

    A = reshape(A, dim_a, dim_b)

    if return_tensor_shape
        return A, shp_a, shp_b
    else
        return A
    end
end

function find_trunc_dim(v, chis, eps,
                        break_degenerate=false, degeneracy_eps=1e-6,
                        norm_type=:frobenius)
    # Put chis in a standard format.
    chis = format_trunc_chis(v, chis, eps)
    v = abs.(v)
    if !issorted(v, rev=true)
        sort!(v, rev=true)
    end

    if norm_type==:frobenius
        v = v.^2
        eps = eps^2
    elseif norm_type == :trace
        # Nothing to be done
    else
        throw(ArgumentError("Unknown norm_type $norm_type."))
    end
    sum_all = sum(v)
    # Find the smallest chi for which the error is small enough.
    # If none is found, use the largest chi.
    if sum_all != 0
        for i in 1:length(chis)
            chi = chis[i]
            if !break_degenerate
                # Make sure that we don't break degenerate singular values by
                # including one but not the other by decreasing chi if
                # necessary.
                while 0 < chi < length(v)
                    last_in = v[chi]
                    last_out = v[chi+1]
                    rel_diff = abs(last_in - last_out)/last_in
                    if rel_diff < degeneracy_eps
                        chi -= 1
                    else
                        break
                    end
                end
            end
            sum_disc = sum(v[chi+1:end])
            error = sum_disc/sum_all
            if error <= eps
                break
            end
        end
        if norm_type==:frobenius
            error = sqrt(error)
        end
    else
        error = 0
        chi = minimum(chis)
    end
    return chi, error
end


function format_trunc_chis(v, chis, eps)
    max_dim = length(v)
    if chis == nothing
        if eps > 0
            # Try all possible chis.
            chis = collect(1:max_dim)
        else
            # No truncation.
            chis = [max_dim]
        end
    else
        if isa(chis, Number)
            # Wrap an individual number in an Array.
            chis = [chis]
        else
            # Make sure chis is an Array, and not, say, a tuple.
            chis = collect(chis)
        end
        if eps == 0
            chis = [maximum(chis)]
        else
            sort!(chis)
        end
    end
    # If some of the chis are larger than max_dim, get rid of them.
    for (i, chi) in enumerate(chis)
        if chi >= max_dim
            chis[i] = max_dim
            chis = chis[1:i]
            break
        end
    end
    return chis
end

In [ ]:
# Function used to test if the tensor of coefficients of the homogenous polynomial used to expand the β parameter is positive definte so 
# the β > 0 condition is maintained in order to have a feasible probability distribution.

function tensor_definiteness(a,vars,degree)
    # Create and prepare the tensor to be evaluated
    tensor_coefs = ones(vars^degree)
            
    
    ind = 1
    for (c,m) in enumerate(eachmultinomial(vars,degree))
        bc = multinomial(m)
        
        if ind === 1 || ind === length(tensor_coefs)
            tensor_coefs[ind] = a[c]
        else
            tensor_coefs[ind:ind+bc-1] = fill(a[c]/bc,bc)
        end
        ind += bc
    end
    
    tensor_coefs = reshape(tensor_coefs,[vars for c=1:degree]...)

    # Find and return the # of negative eigenvalues 
    eigenvalues = tensoreig(tensor_coefs,collect(1:(degree ÷ 2)),collect((degree ÷ 2)+1:degree))[1]
    if !(isa(eigenvalues, Vector{ComplexF64}) || isa(eigenvalues, Vector{ComplexF32}))
        # tensoreig does not provide exact values and so need small window below 0
        return sum(eigenvalues .<= -1.0*10^-6)#-1.0*10^-6
    else
        return sum([c.im !== 0.0 for c in eigenvalues])
    end
end

In [ ]:
# Creation of data and calculation of its empirical CDF to be used to fit

draws = [rand(MvLogNormal([0.,0.],[1. 0.4; 0.4 1.])) for c=1:100]

sorted_draws = sort(draws,by=x->x[1])

MV_ECDF = Dict{Vector{Float64}, Float64}()

l_sorted_draws = length(sorted_draws)
for c=1:length(sorted_draws)
    s = 0.
    for d=1:length(sorted_draws)
        if sorted_draws[c][1] >= sorted_draws[d][1] && sorted_draws[c][2] >= sorted_draws[d][2]
            s += 1.
        end
        
    end
    MV_ECDF[sorted_draws[c]] = s/l_sorted_draws
end

sorted_draws = sort(draws,by=x->x[1])
draws_ECDF = [MV_ECDF[c] for c in draws];

In [ ]:
# Specify the number order of the polynomial expansions and hence the number of terms as well as the
# dimension of the distribution

n_th_order = 4
n_variables = 2


# Find all the combinatorical numbers needed to specify and create the polynomials
bin_coef(n,k) = factorial(n)÷(factorial(k)*factorial(n - k))
simplex_numbers(r,n) = bin_coef(n+r-1,r)
x_medoid = [0.,0.]

l_even_coefficients = [simplex_numbers(n_variables-1,c+1) for c=2:2:n_th_order]
l_odd_coefficients = [simplex_numbers(n_variables-1,c+1) for c=1:2:n_th_order]

n_even_coefficients = n_variables*sum(l_even_coefficients)
n_odd_coefficients = n_variables*sum(l_odd_coefficients)

multinomial_exponents = [[[r for r in k] for k in eachmultinomial(n_variables,n)] for n=1:n_th_order]
multinomial_coefficients = [[multinomial(k) for k in eachmultinomial(n_variables,n)] for n=1:n_th_order];

# Returns the constructed polynomial used for expansion
function MV_param_expansion(x,a,parity,coef=multinomial_coefficients,expo=multinomial_exponents,expan=x_medoid)
    a_i = 0
    a_l = length(a)
    x_l = length(x)
    n = parity == 2 ? 0 : -1
    s = 0.
    while a_i < a_l
        n += 2
        for (i,k) in enumerate(expo[n])
            a_i += 1
            s += coef[n][i]*a[a_i]*prod([(x[c] - expan[c])^(k[c]) for c=1:x_l])
        end
    end
            
    return s
end

In [ ]:
# Functions for the pdf and CDF of the multivariate meta-gumbel distribution

function gumbel_cdf(x,aμ,aβ)
    μ = [MV_param_expansion(x,aμ[c,:],1) for c=1:size(aμ)[1]]
    β = [MV_param_expansion(x,aβ[c,:],2) for c=1:size(aβ)[1]]
    return exp(sum([-exp(-(x[c]-μ[c])/β[c]) for c=1:size(aμ)[1]]))
end

function gumbel_cdf(x,aμ=am,aβ=as)
    μ = [MV_param_expansion(x,aμ[c,:],1) for c=1:size(aμ)[1]]
    β = [MV_param_expansion(x,aβ[c,:],2) for c=1:size(aβ)[1]]
    return exp(sum([-exp(-(x[c]-μ[c])/β[c]) for c=1:size(aμ)[1]]))
end

function gumbel_pdf(x,aμ,aβ)
    μ = [MV_param_expansion(x,aμ[c,:],1) for c=1:size(aμ)[1]]
    β = [MV_param_expansion(x,aβ[c,:],2) for c=1:size(aβ)[1]]
    return exp(sum([-(x[c]-μ[c])/β[c]-exp(-(x[c]-μ[c])/β[c]) for c=1:size(aμ)[1]]))/(prod(β))
end

In [ ]:
# Loss function used in the initial localisation of a feasibility region when optimising the fit

function gumbel_feasible_loss(x,x_l=n_variables,n_ord=n_th_order,n_e=n_even_coefficients,n_o=n_odd_coefficients,l_e=l_even_coefficients,l_o=l_odd_coefficients)
    
    # Array of parameters for the coefficients on the β expansion 
    as = [x[(c-1)*(n_e ÷ 2) + d] for c=1:x_l, d=(n_o+1:(n_o+n_e ÷ 2))]
    
    eigenvalue_sum = 0
    
    cum_sum_even_coef = cumsum(l_e)
    for c=1:size(as)[1]
        order = 0
        for d=0:length(cum_sum_even_coef)-1
            order += 2
            if d === 0
                eigenvalue_sum += tensor_definiteness(as[c,1:cum_sum_even_coef[1]],x_l,order)
            else
                eigenvalue_sum += tensor_definiteness(as[c,cum_sum_even_coef[d]+1:cum_sum_even_coef[d+1]],x_l,order)
            end
        end
    end
    
    # Returns the number of negative eigenvalues of the tensor representing the homogeneous polynomial
    # of the β expansion
    return eigenvalue_sum^2
    
end

# Loss function used to fit the distribution to the empirical CDF of the data

function gumbel_cdf_loss(x,Z=draws_ECDF,x_l=n_variables,n_ord=n_th_order,n_e=n_even_coefficients,n_o=n_odd_coefficients,l_e=l_even_coefficients,l_o=l_odd_coefficients)
    
    # Same first part as gumbel_feasible_loss to ensure that the optimisation algorithm doesn't leave
    # a feasible region upon finding one while fitting the CDF
    
    as = [x[(c-1)*(n_e ÷ 2) + d] for c=1:x_l, d=(n_o+1:(n_o+n_e ÷ 2))]
    
    eigenvalue_sum = 0
    
    cum_sum_even_coef = cumsum(l_e)
    for c=1:size(as)[1]
        order = 0
        for d=0:length(cum_sum_even_coef)-1
            order += 2
            if d === 0
                eigenvalue_sum += tensor_definiteness(as[c,1:cum_sum_even_coef[1]],x_l,order)
            else
                eigenvalue_sum += tensor_definiteness(as[c,cum_sum_even_coef[d]+1:cum_sum_even_coef[d+1]],x_l,order)
            end
        end
    end
    
    if eigenvalue_sum > 0.
        #println(100000*eigenvalue_sum^2)
        return 100000*eigenvalue_sum^2
    end
    

    L = length(x)
    
    am = [x[(c-1)*(n_o ÷ 2) + d] for c=1:x_l, d=1:(n_o ÷ 2)]

    # Returns the square error between the empirical cdf and the current fit
    res = sum(([gumbel_cdf(draws[c],am,as) for c=1:length(draws)] .- Z).^2)
    return res
end

In [ ]:
# Uniformly randomly generated initial point to begin the optimisation at
x₀ = -1500 .+ rand(n_even_coefficients + n_odd_coefficients)*3000

In [ ]:
# Initial optimisation to find a feasible region

res = optimize(gumbel_feasible_loss, x₀, ParticleSwarm(n_particles = 15),
                Optim.Options(iterations=12000))

In [ ]:
println("Number of negative eigenvalues in the β coefficient tensor: "*string(sqrt(Optim.minimum(res))))
initial_viable_point = Optim.minimizer(res) 

In [ ]:
# Optimisation of the metadistribution's CDF to the empirical CDF of the data

res = optimize(gumbel_cdf_loss, initial_viable_point, ParticleSwarm(n_particles = 18),
    Optim.Options(iterations=11000))

In [ ]:
# Square error between the empirical cdf and the current fit
println(Optim.minimum(res))
res_x = Optim.minimizer(res)

<font size="3.5">Example of a $n = 200$ draw from a multivariate LogNormal distribution with </font>$\mathbf{\mu} = \mathbf{0}$, 
$\mathbf{\Sigma} = \begin{pmatrix}
1 & 0.4\\
0.4 & 1
\end{pmatrix}$

![image](l_metalog_scatter.png)

In [ ]:
# Form the arrays of coefficients used by gumbel_cdf and gumbel_pdf
am = [res_x[(c-1)*(n_odd_coefficients ÷ 2) + d] for c=1:n_variables, d=1:(n_odd_coefficients ÷ 2)]
as = [res_x[(c-1)*(n_even_coefficients ÷ 2) + d] for c=1:n_variables, d=(n_odd_coefficients+1:(n_odd_coefficients+n_even_coefficients ÷ 2))]

In [ ]:
# Plot the fitted CDF

surface(0.01:0.01:5.,0.01:0.01:5.,(x,y)->gumbel_cdf([x,y],am,as),camera=(45, 65),legend=nothing,zlim=(0,1),xlabel="X",ylabel="Y",zlabel="F(x,y)",c=:oslo)